Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

C:\Users\o.ivashkin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [33]:
batch_size = 128
reg_ = 0.01

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b1 = tf.Variable(tf.zeros([num_labels]))
    
    l_out = tf.matmul(tf_train_dataset, w1) + b1
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=l_out) ) + reg_ * tf.nn.l2_loss(w1)
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    train_prediction = tf.nn.softmax(l_out)
    
        
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, w1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, w1) + b1)
    

In [34]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 49.140015
Minibatch accuracy: 7.8%
Validation accuracy: 8.8%
Minibatch loss at step 500: 12.315790
Minibatch accuracy: 69.5%
Validation accuracy: 72.8%
Minibatch loss at step 1000: 4.551421
Minibatch accuracy: 75.8%
Validation accuracy: 77.2%
Minibatch loss at step 1500: 2.084340
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 1.257169
Minibatch accuracy: 82.0%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 1.040082
Minibatch accuracy: 76.6%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 0.801841
Minibatch accuracy: 82.8%
Validation accuracy: 82.5%
Test accuracy: 89.5%


In [4]:
n_hidden = 1024

In [25]:
batch_size = 128
reg_ = 0.008

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
    b1 = tf.Variable(tf.zeros([n_hidden]))

    w2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    l_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    l_out = tf.matmul(l_hidden, w2) + b2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=l_out) ) + reg_ * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) )
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    train_prediction = tf.nn.softmax(l_out)
    
        
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)
    

In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2899.657715
Minibatch accuracy: 7.0%
Validation accuracy: 27.6%
Minibatch loss at step 500: 1134.889038
Minibatch accuracy: 74.2%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 504.342712
Minibatch accuracy: 85.2%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 226.128876
Minibatch accuracy: 78.9%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 101.325371
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 2500: 45.869400
Minibatch accuracy: 84.4%
Validation accuracy: 86.8%
Minibatch loss at step 3000: 20.865299
Minibatch accuracy: 87.5%
Validation accuracy: 86.5%
Test accuracy: 92.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [6]:
def drawsamp(data, labels, n):
    indices = np.random.choice(range(len(data)), size = n, replace=False, p=None)
    return data[indices], labels[indices], indices

In [40]:
train_data_samp, train_labels_samp, _samp = drawsamp(train_dataset, train_labels, 128 * 10)

In [58]:
reg_ = 0.0

In [42]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_samp.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_samp[offset:(offset + batch_size), :]
        batch_labels = train_labels_samp[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 313.404785
Minibatch accuracy: 13.3%
Validation accuracy: 29.2%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Minibatch loss at step 1500: 0.000001
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.5%
Test accuracy: 80.9%


Oleg:
Minibatch loss = 0 and Minibatch accuracy = 100% tell us than the model has been ideally fitted to training set. However, Validation accuracy and Test accuracy are substantially lower on test and 

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [56]:
n_hidden = 1024
batch_size = 128
prob = 1/2

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
    b1 = tf.Variable(tf.zeros([n_hidden]))

    w2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    l_hidden = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    l_out = tf.matmul(tf.nn.dropout(l_hidden, keep_prob = prob), w2) + b2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=l_out) )
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    train_prediction = tf.nn.softmax(l_out)
    
        
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)
    

In [60]:
train_data_samp, train_labels_samp, _samp = drawsamp(train_dataset, train_labels, 200000)

In [61]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_samp.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_samp[offset:(offset + batch_size), :]
        batch_labels = train_labels_samp[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 549.196838
Minibatch accuracy: 14.1%
Validation accuracy: 24.5%
Minibatch loss at step 500: 33.213791
Minibatch accuracy: 78.1%
Validation accuracy: 82.6%
Minibatch loss at step 1000: 31.152176
Minibatch accuracy: 77.3%
Validation accuracy: 83.5%
Minibatch loss at step 1500: 14.552491
Minibatch accuracy: 74.2%
Validation accuracy: 82.5%
Minibatch loss at step 2000: 6.043719
Minibatch accuracy: 79.7%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 6.461442
Minibatch accuracy: 78.9%
Validation accuracy: 83.3%
Minibatch loss at step 3000: 3.561191
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Test accuracy: 88.7%


Oleg: let's explore overfitting problem (below)

In [55]:
train_data_samp, train_labels_samp, _samp = drawsamp(train_dataset, train_labels, 128 * 5)

In [57]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_samp.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_samp[offset:(offset + batch_size), :]
        batch_labels = train_labels_samp[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 524.805786
Minibatch accuracy: 11.7%
Validation accuracy: 24.8%
Minibatch loss at step 500: 0.117071
Minibatch accuracy: 99.2%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 0.165751
Minibatch accuracy: 99.2%
Validation accuracy: 78.0%
Minibatch loss at step 1500: 0.380588
Minibatch accuracy: 98.4%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.4%
Test accuracy: 85.9%


Oleg: Validation accuracy and Test accuracy are somewhat higher than with no reularization.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [7]:
train_data_samp, train_labels_samp, _samp = drawsamp(train_dataset, train_labels, 200*1000)

In [34]:
n_hidden_1 = 64
n_hidden_2 = 46
n_hidden_3 = 32
n_hidden_4 = 16
n_hidden_5 = 16

batch_size = 1024
prob = 1/2

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_1]))
    b1 = tf.Variable(tf.zeros([n_hidden_1]))
    
    w2 = tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2]))
    b2 = tf.Variable(tf.zeros([n_hidden_2]))
    
    w3 = tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3]))
    b3 = tf.Variable(tf.zeros([n_hidden_3]))
    
    w4 = tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4]))
    b4 = tf.Variable(tf.zeros([n_hidden_4]))
    
    w5 = tf.Variable(tf.truncated_normal([n_hidden_4, n_hidden_5]))
    b5 = tf.Variable(tf.zeros([n_hidden_5]))
    
    w6 = tf.Variable(tf.truncated_normal([n_hidden_5, num_labels]))
    b6 = tf.Variable(tf.zeros([num_labels]))
    
    
    def model(data):
        hidden_1 = tf.tanh(tf.matmul(data, w1) + b1)
        hidden_2 = tf.tanh(tf.matmul(tf.nn.dropout(hidden_1, keep_prob = prob), w2) + b2)
        hidden_3 = tf.tanh(tf.matmul(tf.nn.dropout(hidden_2, keep_prob = prob), w3) + b3)
        hidden_4 = tf.tanh(tf.matmul(tf.nn.dropout(hidden_3, keep_prob = 1), w4) + b4)
        hidden_5 = tf.tanh(tf.matmul(tf.nn.dropout(hidden_4, keep_prob = 1), w5) + b5)
        return tf.matmul(tf.nn.dropout(hidden_5, keep_prob = 1), w6) + b6
    
    train_out = model(tf_train_dataset)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits = train_out) )
    
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    
    train_prediction = tf.nn.softmax(train_out)
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset) )
    test_prediction = tf.nn.softmax(model(tf_test_dataset) )

In [35]:
num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_samp.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_samp[offset:(offset + batch_size), :]
        batch_labels = train_labels_samp[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.609995
Minibatch accuracy: 10.5%
Validation accuracy: 10.7%
Minibatch loss at step 500: 2.344069
Minibatch accuracy: 15.2%
Validation accuracy: 15.8%
Minibatch loss at step 1000: 2.186875
Minibatch accuracy: 19.7%
Validation accuracy: 19.5%
Minibatch loss at step 1500: 2.133749
Minibatch accuracy: 24.7%
Validation accuracy: 23.8%
Minibatch loss at step 2000: 2.024539
Minibatch accuracy: 25.0%
Validation accuracy: 27.5%
Minibatch loss at step 2500: 1.943604
Minibatch accuracy: 29.5%
Validation accuracy: 29.8%
Minibatch loss at step 3000: 1.834243
Minibatch accuracy: 33.9%
Validation accuracy: 33.1%
Minibatch loss at step 3500: 1.841804
Minibatch accuracy: 35.4%
Validation accuracy: 35.2%
Minibatch loss at step 4000: 1.753429
Minibatch accuracy: 38.0%
Validation accuracy: 37.7%
Minibatch loss at step 4500: 1.746437
Minibatch accuracy: 39.4%
Validation accuracy: 38.8%
Minibatch loss at step 5000: 1.730844
Minibatch accuracy: 40.2%
Validation accurac